In [1]:
import env
import numpy as np
import pandas as pd

Acquire data from mySQL using the python module to connect and query. 
**Objective: create acquire.py**

- single dataframe

- content requirements:
> logerror
>
> all fields
>
> airconditioningtypeid
>
> all properties having transaction in 2017, 
>
> include only the last transaction for each properity = no duplicate property id's
>
> zestimate error 
>
> date of transaction
> 
> only include properties that include a latitude and longitude value